<a href="https://colab.research.google.com/github/Oliivkaaaa/BigDate_/blob/main/MongoDB_%D0%9D%D0%B5%D1%82%D0%B8%D0%BF%D0%B8%D1%87%D0%BD%D0%B0%D1%8F_%D1%81%D1%85%D0%B5%D0%BC%D0%B0_%D0%B8_%D0%B7%D0%B0%D0%BF%D1%80%D0%BE%D1%81%D1%8B%C2%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install mongomock pymongo


import mongomock
import pprint
from pymongo import MongoClient

client = mongomock.MongoClient()
db = client["dream_db"]
collection = db["dream_journal"]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 22.5 MB/s eta 0:00:00


In [7]:
client = mongomock.MongoClient()
db = client["dream_db"]
collection = db["dream_journal"]

doc = {
    "user_id": 101,
    "username": "alice",
    "dreams": [
        {
            "date": "20.10.2025",
            "title": "Полет над городом",
            "mood": "inspired",
            "symbols": [
                {"name": "sky", "meaning": "freedom"},
                {"name": "city", "meaning": "ambition"}
            ],
            "characters": [
                {"name": "bird", "type": "animal", "feeling": "friendly"}
            ],
            "notes": {"lucid": True, "clarity": 8, "themes": ["flight", "freedom"]}
        },
        {
            "date": "20.10.2025",
            "title": "Лес и тень",
            "mood": "fear",
            "symbols": [
                {"name": "forest", "meaning": "unknown"},
                {"name": "shadow", "meaning": "hidden fears"}
            ],
            "characters": [
                {"name": "shadow", "type": "entity", "feeling": "hostile"}
            ],
            "notes": {"lucid": False, "clarity": 5, "themes": ["darkness", "fear"]}
        }
    ]
}

collection.insert_one(doc)

In [8]:
print("\n=== Запрос по вложенному полю (dreams.symbols.name == 'shadow') ===")
query = {"dreams.symbols.name": "shadow"}
projection = {"username": 1, "dreams.title": 1}
for doc in collection.find(query, projection):
    pprint.pprint(doc)


=== Запрос по вложенному полю (dreams.symbols.name == 'shadow') ===
{'_id': ObjectId(aea68868-afdb-11f0-b792-0242ac1c000c),
 'dreams': [{'title': 'Полет над городом'}, {'title': 'Лес и тень'}],
 'username': 'alice'}


In [9]:
print("\n=== Агрегация: количество снов по настроению (mood) ===")
pipeline = [
    {"$unwind": "$dreams"},
    {"$group": {"_id": "$dreams.mood", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]
for doc in collection.aggregate(pipeline):
    pprint.pprint(doc)


=== Агрегация: количество снов по настроению (mood) ===
{'_id': 'fear', 'count': 1}
{'_id': 'inspired', 'count': 1}


In [10]:
print("\n=== Модификация массива: добавляем новый символ в сон 'Лес и тень' ===")
doc = collection.find_one({"dreams.title": "Лес и тень"})
for dream in doc["dreams"]:
    if dream["title"] == "Лес и тень":
        dream["symbols"].append({"name": "door", "meaning": "new opportunity"})
collection.replace_one({"_id": doc["_id"]}, doc)

print("\n=== Проверка обновлённого документа ===")
updated = collection.find_one({"username": "alice"}, {"dreams": 1})
pprint.pprint(updated)



=== Модификация массива: добавляем новый символ в сон 'Лес и тень' ===

=== Проверка обновлённого документа ===
{'_id': ObjectId(aea68868-afdb-11f0-b792-0242ac1c000c),
 'dreams': [{'characters': [{'feeling': 'friendly',
                             'name': 'bird',
                             'type': 'animal'}],
             'date': '20.10.2025',
             'mood': 'inspired',
             'notes': {'clarity': 8,
                       'lucid': True,
                       'themes': ['flight', 'freedom']},
             'symbols': [{'meaning': 'freedom', 'name': 'sky'},
                         {'meaning': 'ambition', 'name': 'city'}],
             'title': 'Полет над городом'},
            {'characters': [{'feeling': 'hostile',
                             'name': 'shadow',
                             'type': 'entity'}],
             'date': '20.10.2025',
             'mood': 'fear',
             'notes': {'clarity': 5,
                       'lucid': False,
                       